In [ ]:
%%capture
import numpy as np
import MCTS
import naiveRule
%config InlineBackend.figure_formats = ['svg']

In [ ]:
# MCTS.a0eng2.a0_eng.batnor91.variables

In [ ]:
npz_t=np.load("./games/dat_train1.npz")
x_tr=npz_t['arr_0']
y_tr=npz_t['arr_1']
print(len(y_tr))
print(np.average(y_tr[:,-1]))

In [ ]:
i=0

In [ ]:
MCTS.showHeatMap(x_tr[i],y_tr[i],MCTS.a0eng.a0_eng(np.array([x_tr[i]]))[0].numpy())
i+=1

In [ ]:
i-=2

In [ ]:
y_tr[i]

In [ ]:
xt=np.zeros([15,15,2])
xt[5,7,0]=1
xt[6,8,1]=1
xt[6,6,0]=1
xt[4,8,1]=1
xt[7,7,0]=1
xt[5,8,1]=1
xt[3,8,0]=1
xt[4,7,1]=1
xt[6,9,0]=1
xt[4,9,1]=1
xt[4,10,0]=1
xt[4,5,1]=1
xt[4,6,0]=1
xt[6,7,1]=1
xt[7,6,0]=1
xt=np.flip(xt,-1)

In [ ]:
MCTS.showHeatMap(xt,MCTS.a0eng.a0_eng(np.array([xt]))[0].numpy(),MCTS.a0eng2.a0_eng(np.array([xt]))[0].numpy())

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%config InlineBackend.figure_formats = ['svg']
fp=open("trLog.log","r")
lsv=[]
pv=[]
vv=[]
lst=[]
pt=[]
vt=[]
rr=[]
dbl=[]
while(True):
    ss=fp.readline()
    if(ss==""):
        break
    es=ss.split()
    if("training" in es):
        vv+=[float(es[-1])]
        pv+=[float(es[-3])]
        lsv+=[float(es[-5])]
        vt+=[float(es[-7])]
        pt+=[float(es[-9])]
        lst+=[float(es[-11])]
    elif("dbls" in es):
        rr+=[float(es[4])]
        dbl+=[int(es[7])]
fp.close()

elo=[]
elot=[]
elo_teacher=1500.0
def rrt2elo(rat,el_tr):
    rrat=min(max(0.05,rat),0.95)
    return el_tr-np.log10((1-rrat)/rrat)*400
for i in range(len(dbl)):
    elo+=[rrt2elo(rr[i]/64,elo_teacher)]
    elot+=[elo_teacher]
    if(rr[i]-dbl[i]/2>32):
        elo_teacher=elo[-1]

_, ((ax1,ax2),(ax3,ax4)) = plt.subplots(2,2,figsize=(7/.618,7))
ax1.plot(lst)
ax1.plot(lsv)
ax1.grid(which='both')
ax1.set_title("loss")
ax2.plot(pt)
ax2.plot(pv)
ax2.grid(which='both')
ax2.set_title("p accuracy")
ax3.plot(vt)
ax3.plot(vv)
ax3.grid(which='both')
ax3.set_title("v accuracy")
ax4.plot(elot)
ax4.plot(elo)
ax4.grid(which='both')
ax4.set_title("elo rating")
plt.show()

In [ ]:
if(os.path.isfile("./games/dat_trprev.npz")):
    npz_t=np.load("./games/dat_trprev.npz")
    x_tr=npz_t['arr_0']
    y_tr=npz_t['arr_1']
    print("load cached data: %d"%(len(y_tr)))
    npz_t=np.load("./games/dat_train1.npz")
    x_tr=np.concatenate((x_tr,npz_t['arr_0']))
    y_tr=np.concatenate((y_tr,npz_t['arr_1']))
else:
    npz_t=np.load("./games/dat_train1.npz")
    x_tr=npz_t['arr_0']
    y_tr=npz_t['arr_1']

npz_v=np.load("./games/dat_vlidn1.npz")
x_vl=npz_v['arr_0']
y_vl=npz_v['arr_1']

for ii in range(2,33):
    nf="./games/dat_train"+str(ii)+".npz"
    if(os.path.exists(nf)):
        npz_t=np.load(nf)
        x_tr=np.concatenate((x_tr,npz_t['arr_0']))
        y_tr=np.concatenate((y_tr,npz_t['arr_1']))
    nf="./games/dat_vlidn"+str(ii)+".npz"
    if(os.path.exists(nf)):
        npz_v=np.load(nf)
        x_vl=np.concatenate((x_vl,npz_v['arr_0']))
        y_vl=np.concatenate((y_vl,npz_v['arr_1']))

In [ ]:
bz=256
sg=np.std(x_tr,axis=(0,1,2))
mu=np.mean(x_tr,axis=(0,1,2))
for i in range(25):
    bsg=np.std(x_tr[bz*i:bz*(i+1)],axis=(0,1,2))
    bmu=np.mean(x_tr[bz*i:bz*(i+1)],axis=(0,1,2))
    print(bsg/sg,(bmu-mu)/sg)
    if(bz*(i+1)>len(y_tr)):
        break